## Instruction;
1. target; is_safe
2. feature; ammonia, lead, baterias, viruses, nitrates, uranium
3. Missing value treatment 
4. Multicollinearty checking
5. Data split to train:test = 80% : 20%, random_state=0, stratify=y
6. Create model, test the accuracy of the model to predict

In [2]:
import numpy as np
import pandas as pd

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor # to calculate VIF value

from sklearn.model_selection import train_test_split # to split dataset randomly
from sklearn.metrics import accuracy_score # to calculate the accuracy score

import warnings
warnings.filterwarnings("ignore") # to remove warning when running cell

In [16]:
df = pd.read_csv('waterQuality1.csv')
df.head()

,aluminium,ammonia,arsenic,barium,cadmium,chloramine,chromium,copper,flouride,bacteria,...,lead,nitrates,nitrites,mercury,perchlorate,radium,selenium,silver,uranium,is_safe
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,...,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,...,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,...,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,...,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,...,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1


### Description

All attributes are numeric variables and they are listed bellow:

1. aluminium - dangerous if greater than 2.8
2. ammonia - dangerous if greater than 32.5
3. arsenic - dangerous if greater than 0.01
4. barium - dangerous if greater than 2
5. cadmium - dangerous if greater than 0.005
6. chloramine - dangerous if greater than 4
7. chromium - dangerous if greater than 0.1
8. copper - dangerous if greater than 1.3
9. flouride - dangerous if greater than 1.5
10. bacteria - dangerous if greater than 0
11. viruses - dangerous if greater than 0
12. lead - dangerous if greater than 0.015
13. nitrates - dangerous if greater than 10
14. nitrites - dangerous if greater than 1
15. mercury - dangerous if greater than 0.002
16. perchlorate - dangerous if greater than 56
17. radium - dangerous if greater than 5
18. selenium - dangerous if greater than 0.5
19. silver - dangerous if greater than 0.1
20. uranium - dangerous if greater than 0.3
21. is_safe - class attribute {0 - not safe, 1 - safe}

dataset taken from kaggle; https://www.kaggle.com/mssmartypants/water-quality

In [20]:
df = df[['ammonia', 'lead', 'bacteria', 'viruses', 'nitrates', 'uranium', 'is_safe']]
df.head()

,ammonia,lead,bacteria,viruses,nitrates,uranium,is_safe
0,9.08,0.054,0.20,0.000,16.08,0.02,1
1,21.16,0.100,0.65,0.650,2.01,0.05,1
2,14.02,0.078,0.05,0.003,14.16,0.01,0
3,11.33,0.016,0.71,0.710,1.41,0.05,1
4,24.33,0.117,0.13,0.001,6.74,0.02,1


#### Analyze the correlation of lead to the quality of water

In [21]:
df[['ammonia', 'lead', 'bacteria', 'viruses', 'nitrates', 'uranium', 'is_safe']].describe(include='all') ## Include all because the is_safe column's datatypes is object

,ammonia,lead,bacteria,viruses,nitrates,uranium,is_safe
count,7999,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999
unique,2564,NaN,NaN,NaN,NaN,NaN,3
top,0.17,NaN,NaN,NaN,NaN,NaN,0
freq,13,NaN,NaN,NaN,NaN,NaN,7084
mean,NaN,0.099450,0.319665,0.328583,9.818822,0.044673,NaN
std,NaN,0.058172,0.329485,0.378096,5.541331,0.026904,NaN
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,NaN,0.048000,0.000000,0.002000,5.000000,0.020000,NaN
50%,NaN,0.102000,0.220000,0.008000,9.930000,0.050000,NaN
75%,NaN,0.151000,0.610000,0.700000,14.610000,0.070000,NaN


#### EDA

In [23]:
df.isna().sum()

ammonia     0
lead        0
bacteria    0
viruses     0
nitrates    0
uranium     0
is_safe     0
dtype: int64

In [24]:
df[df['is_safe']=='#NUM!']

,ammonia,lead,bacteria,viruses,nitrates,uranium,is_safe
7551,#NUM!,0.183,0.00,0.0,4.37,0.05,#NUM!
7568,#NUM!,0.178,0.00,0.0,12.10,0.07,#NUM!
7890,#NUM!,0.088,0.57,0.0,9.57,0.02,#NUM!


In [25]:
df = df.drop(index=[7551,7568,7890], axis=0) ## We drop rows that contains #NUM! value in the is_safe column. drop is chosen because the affected row is little

In [26]:
df['is_safe'].value_counts()

0    7084
1     912
Name: is_safe, dtype: int64

In [27]:
df[['lead', 'is_safe']].describe(include='all')

,lead,is_safe
count,7996.000000,7996
unique,NaN,2
top,NaN,0
freq,NaN,7084
mean,0.099431,NaN
std,0.058169,NaN
min,0.000000,NaN
25%,0.048000,NaN
50%,0.102000,NaN
75%,0.151000,NaN


In [31]:
df['is_safe']=df['is_safe'].astype(float) # Casting is_safe dtypes to float

In [49]:
df['ammonia']=df['ammonia'].astype(float) # Casting ammonia dtypes to float

In [44]:
df[['ammonia', 'lead', 'bacteria', 'viruses', 'nitrates', 'uranium', 'is_safe']].corr()

,ammonia,lead,bacteria,viruses,nitrates,uranium,is_safe
ammonia,1.000000,-0.037065,0.063603,0.105856,0.006483,0.014635,-0.022919
lead,-0.037065,1.000000,-0.027179,0.017886,0.035095,-0.009372,-0.009970
bacteria,0.063603,-0.027179,1.000000,0.618535,-0.033993,0.045077,-0.022077
viruses,0.105856,0.017886,0.618535,1.000000,-0.044621,0.058473,-0.097040
nitrates,0.006483,0.035095,-0.033993,-0.044621,1.000000,0.000795,-0.072100
uranium,0.014635,-0.009372,0.045077,0.058473,0.000795,1.000000,-0.075619
is_safe,-0.022919,-0.009970,-0.022077,-0.097040,-0.072100,-0.075619,1.000000


### The correlation between each feature to target is figured above. From the table we can conclude that 'viruses' have highest correlation to the target. the value is negative, meaning that the correlation is contradictive.

## Difining X and Y

In [45]:
X = df[['ammonia', 'lead', 'bacteria', 'viruses', 'nitrates', 'uranium']]
y = df['is_safe']

In [46]:
X

,ammonia,lead,bacteria,viruses,nitrates,uranium
0,9.08,0.054,0.20,0.000,16.08,0.02
1,21.16,0.100,0.65,0.650,2.01,0.05
2,14.02,0.078,0.05,0.003,14.16,0.01
3,11.33,0.016,0.71,0.710,1.41,0.05
4,24.33,0.117,0.13,0.001,6.74,0.02
...,...,...,...,...,...,...
7994,7.78,0.197,0.00,0.000,14.29,0.03
7995,24.22,0.031,0.00,0.000,10.27,0.08
7996,6.85,0.182,0.00,0.000,15.92,0.05
7997,10.00,0.000,0.00,0.000,0.00,0.00


### Multicollinearity check

In [47]:
# Function to calculate VIF
def calc_vif(X):
    vif = pd.DataFrame()
    vif['variables'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['Acceptable'] = np.where(vif.VIF < 4, 'Yes', 'No') 
    return (vif)

In [48]:
calc_vif(X)

,variables,VIF,Acceptable
0,ammonia,2.961646,Yes
1,lead,3.041048,Yes
2,bacteria,3.056452,Yes
3,viruses,2.875015,Yes
4,nitrates,3.208677,Yes
5,uranium,3.033962,Yes


#### All variables have VIF value less than 4 whch is good

## Data Splitting

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    stratify=y,
    random_state = 0,
    test_size= 0.2
)

In [53]:
print('raw data shape is;',X.shape)
print('X_train data shape is;', X_train.shape)
print('X_test raw data shape is;',X_test.shape)

raw data shape is; (7996, 6)
X_train data shape is; (6396, 6)
X_test raw data shape is; (1600, 6)


## **Logistic regression modelling using statsmodels**

In [54]:
sm_logit = sm.Logit(y_train, sm.add_constant(X_train))
result = sm_logit.fit()

Optimization terminated successfully.
         Current function value: 0.343578
         Iterations 7


In [55]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                is_safe   No. Observations:                 6396
Model:                          Logit   Df Residuals:                     6389
Method:                           MLE   Df Model:                            6
Date:                Tue, 12 Oct 2021   Pseudo R-squ.:                 0.03237
Time:                        01:29:20   Log-Likelihood:                -2197.5
converged:                       True   LL-Null:                       -2271.0
Covariance Type:            nonrobust   LLR p-value:                 3.284e-29
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.0644      0.138     -7.727      0.000      -1.334      -0.794
ammonia       -0.0038      0.005     -0.840      0.401      -0.013       0.005
lead          -0.3611      0.691     -0.522      0.6

In [57]:
df.describe().loc[['min', 'max']][['ammonia', 'lead', 'bacteria','viruses','nitrates','uranium']]

,ammonia,lead,bacteria,viruses,nitrates,uranium
min,-0.08,0.0,0.0,0.0,0.00,0.00
max,29.84,0.2,1.0,1.0,19.83,0.09


**The model should calculate is the quality of the water is safe or not**

1. LLR p-value = 3.284e-29 = 0.0000...3284

   - LLR p-value < 0.05, we reject the H0.  Can be say that there is minimum one variable that significantly affect the quality of the water.
<br><br>

2. P>|z| (Wald test)

   - const = 0.000. p-value < 0.05, reject H0. It refers to that the model require intercept.
   - ammonia = 0.401. p-value > 0.05, failed to reject H0. It refers to that the amount of ammonia is not significantly affected the quality of the water.
   - lead = 0.601. p-value > 0.05, failed to reject H0. It refers to that the amount of lead is not significantly affected the quality of the water.
   - bacteria = 0.000. p-value < 0.05, reject H0. It refers to that the amount of bacteria significantly affected the quality of the water in positive manner(the higher the value of bacteria, the higher quality of the water).
   - viruses = 0.000. p-value < 0.05, reject H0. It refers to that the amount of viruses significantly affected the quality of the water in negative manner(the higher the value of ammonia, the lower quality of the water).
   - nitrates = 0.000. p-value > 0.05, reject H0. It refers to that the amount of nitrates significantly affected the quality of the water in negative manner(the higher the value of nitrates, the lower quality of the water).
   - uranium = 0.000. p-value > 0.05, reject H0. It refers to that the amount of uranium significantly affected the quality of the water in negative manner(the higher the value of uranium, the lower quality of the water).
<br><br>

3. Coef
   - const = 1.0644
   - ammonia = -0.0038 (should not be interpreted because the significance to the quality of water is low)
   - lead = -0.3611
   - bacteria = 0.7694 
   - viruses = -1.2544
   - nitrates = -0.0420
   - uranium = -9.2263

## ***Coef intepretation using Odd Ratio (OR)***

OR = exp(beta(c-a))

- If OR > 1, c > a: success rate increase If Xi increase.
- If OR < 1, c > a: success rate decrease If Xi decrease.

### 1. lead OR intepretation

In [59]:
c = 0.2
a = 0.1 # chosse a and c value from range 0-2 with c > a.
Beta = -0.3611

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 0.9645341888866997
OR_pclass_interpretation = 1.0367698849060356


- OR_pclass < 1, success rate increased when Xi (lead) deceased (0.2 to 0.1).
- OR_pclass < 1, water containing lead value of 0.1, can affect the better quality of water for 1.036% than the water containing lead value of 0.2.

### 2. bacteria OR intepretation

In [60]:
c = 1
a = 0 
Beta = 0.7694

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 2.1584707827128295
OR_pclass_interpretation = 0.4632909595112381


- OR_pclass > 1, success rate increased when Xi (bacteria) increased (0 to 1).
- OR_pclass > 1, water containing bacteria value of 1, can affect the better quality of water for 0.46% than the water containing bacteria value of 0.

### 3. viruses OR intepretation

In [61]:
c = 1
a = 0 
Beta = -1.2544

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 0.2852469450573052
OR_pclass_interpretation = 3.505734302602621


- OR_pclass < 1, success rate increased when Xi (viruses) deceased (1 to 0).
- OR_pclass < 1, water containing viruses value of 0, can affect the better quality of water by 3.50% than the water containing viruses value of 1.

### 4. nitrates OR intepretation

In [62]:
c = 6
a = 3 
Beta = -0.0420

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 0.8816148467834161
OR_pclass_interpretation = 1.1342821682830249


- OR_pclass < 1, success rate increased when Xi (nitrates) deceased (6 to 3).
- OR_pclass < 1, water containing nitates value of 3, can affect the better quality of water by 1.13% than the water containing nitrates value of 6.

### 5. uranium OR intepretation

In [63]:
c = 0.09
a = 0.03
Beta = -9.2263

OR_pclass = np.exp(Beta*(c-a))
print('OR_pclass =', OR_pclass)

OR_pclass_interpretation = 1/OR_pclass
print('OR_pclass_interpretation =', OR_pclass_interpretation)

OR_pclass = 0.5748891726382371
OR_pclass_interpretation = 1.7394657050347235


- OR_pclass < 1, success rate increased when Xi (uranium) deceased (0.09 to 0.03).
- OR_pclass < 1, water containing uranium value of 0.03, can affect the better quality of water by 1.73% than the water containing uranium value of 0.09.

### Validate the model using test set

In [65]:
y_predict_proba = result.predict(sm.add_constant(X_test))
y_predict_class = np.where(y_predict_proba > .5, 1, 0)

In [66]:
y_predict_proba

6573    0.104260
6590    0.053403
639     0.087808
3947    0.144735
6539    0.080900
          ...   
3914    0.123151
4066    0.073580
5653    0.155686
515     0.161720
5690    0.106459
Length: 1600, dtype: float64

### Evaluation metrics

In [67]:
print('Model accuracy score in the test set:', accuracy_score(y_test, y_predict_class))

Model accuracy score in the test set: 0.88625


##### From the accuracy score we can conclude that from 100 sample of water test using model, 88% sample will predicted correctly(is_safe or not).

## Check the accuracy of the model using sklearn

In [68]:
from sklearn.linear_model import LogisticRegression

In [69]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [70]:
accuracy_score(y_test, model.predict(X_test))

0.88625

##### In this test, modelling using both sklearn and statsmodel resulting in the same score of 88%